<a href="https://colab.research.google.com/github/kunalnischal7/CustomerChurnPrediction/blob/main/CourierAI_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import drive
from sqlalchemy import create_engine, text

## Google Gemini API Key

In [2]:
genai.configure(api_key="Your-API-Key")

## Loading the dataset from google drive

# Data Preprocessing

In [3]:
drive.mount('/content/drive')
CSV_FILE = '/content/drive/MyDrive/Article_Facility_Dataset.csv'
print("Loading dataset...")
df = pd.read_csv(CSV_FILE)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading dataset...


In [4]:
df

,Article_ID,Facility,Start_Timestamp,End_Timestamp
0,1,Lajpat Nagar,2:16:49,21:01:33
1,1,Vikaspuri,21:01:33,11:49:05
2,1,Saket,11:49:05,2:01:41
3,1,Rohini,2:01:41,18:16:36
4,1,Janakpuri,18:16:36,12:36:30
...,...,...,...,...
3971,1000,Pitampura,3:12:52,8:07:13
3972,1000,Janakpuri,8:07:13,7:38:06
3973,1000,Lajpat Nagar,7:38:06,1:37:13
3974,1000,Rohini,1:37:13,11:48:23


## Ensuring the data has all the columns

In [5]:
required_columns = {'Article_ID', 'Facility', 'Start_Timestamp', 'End_Timestamp'}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Dataset must contain columns: {required_columns}")

# SQL

## Initialising SQL Alchemy Engine

In [6]:
# Initialize the SQLAlchemy engine
from sqlalchemy.types import String
engine = create_engine('sqlite:///:memory:')  # Using an in-memory SQLite database for simplicity

# Create the SQL table
df.to_sql('articles', con=engine, if_exists='replace', index=False, dtype={
    'Start_Timestamp': String(8),  # Explicit length for HH:MM:SS
    'End_Timestamp': String(8)
})
# Debug: Query the table to verify stored timestamps
with engine.connect() as conn:
    result = conn.execute(text("SELECT Start_Timestamp FROM articles WHERE Article_ID = 1"))
    rows = result.fetchall()
    print("Stored Start_Timestamp in SQL table:")
    for row in rows:
        print(row[0])

Stored Start_Timestamp in SQL table:
2:16:49
21:01:33
11:49:05
2:01:41
18:16:36


## Create SQL Database

In [7]:
def generate_sql(natural_query):
    prompt = f"""
    **Task**: Convert natural language to precise SQL
    **Schema**:
    - articles (Article_ID INT, Facility TEXT, Start_Timestamp TEXT, End_Timestamp TEXT)

    **Rules**:
    1. Use EXACT facility names (case-sensitive): {sorted(df['Facility'].unique())}
    2. For counts, use COUNT(DISTINCT Article_ID)
    3. Use GROUP BY for facility-level aggregations
    4. Use ORDER BY and LIMIT for "maximum" or "highest" queries
    5. For timestamp queries, return all matching timestamps unless specified otherwise.

    **Examples**:
    - "How many unique articles passed through Lajpat Nagar?" →
      SELECT COUNT(DISTINCT Article_ID)
      FROM articles
      WHERE Facility='Lajpat Nagar'

    - "Facility with the most articles" →
      SELECT Facility, COUNT(DISTINCT Article_ID) AS Article_Count
      FROM articles
      GROUP BY Facility
      ORDER BY Article_Count DESC
      LIMIT 1

    - "Start timestamp for Article 1" →
      SELECT Start_Timestamp
      FROM articles
      WHERE Article_ID = 1

    **Query to Convert**: "{natural_query}"
    """

    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    return response.text.strip().replace('```sql', '').replace('```', '').strip()

## SQL Validation Layer

In [8]:
def validate_sql(sql):
    """Check for critical components"""
    required_terms = {'SELECT', 'FROM', 'ARTICLES'}  # Uppercase to match case-insensitive check
    if 'GROUP BY' in sql.upper() and 'COUNT(' not in sql.upper():
        raise ValueError("Aggregation queries must include COUNT(DISTINCT Article_ID)")
    return all(term in sql.upper() for term in required_terms)

## Executing SQL

In [9]:
def execute_sql(sql_query):
    """Execute SQL query and return results with enhanced error handling"""
    try:
        # Validate SQL query for safety
        if not sql_query.strip().upper().startswith("SELECT"):
            raise ValueError("Only SELECT queries are allowed for security reasons.")

        # Check for common SQL injection patterns
        if any(keyword in sql_query.upper() for keyword in ["DROP", "DELETE", "UPDATE", "INSERT", "ALTER"]):
            raise ValueError("Modification queries are not permitted.")

        # Execute the query
        with engine.connect() as conn:
            result = conn.execute(text(sql_query))
            rows = result.fetchall()

            # Handle empty results
            if not rows:
                return None  # Return None for no results

            # Always return rows as lists of tuples
            return rows  # Remove single-column special case

    except Exception as e:
        return f"Error executing SQL: {str(e)}"

## Explain SQL Results

In [10]:
# Convert timestamps to HH:MM:SS format
df['Start_Timestamp'] = pd.to_datetime(
    df['Start_Timestamp'],
    format='%H:%M:%S',
    errors='coerce'
).dt.strftime('%H:%M:%S')

df['End_Timestamp'] = pd.to_datetime(
    df['End_Timestamp'],
    format='%H:%M:%S',
    errors='coerce'
).dt.strftime('%H:%M:%S')

# Debug: Print the first few rows to verify
print("Start_Timestamp after preprocessing:")
print(df['Start_Timestamp'].head())

Start_Timestamp after preprocessing:
0    02:16:49
1    21:01:33
2    11:49:05
3    02:01:41
4    18:16:36
Name: Start_Timestamp, dtype: object


In [11]:
def explain_results(results, original_query, sql):
    explanation = ""

    # Handle count results
    if 'COUNT' in sql.upper():
        explanation = f"Found {results[0][0]} unique articles."

    # Handle facility ranking results
    elif 'ORDER BY' in sql.upper() and 'LIMIT 1' in sql.upper():
        facility, count = results[0]
        explanation = f"'{facility}' has the most unique articles: {count}"

    # Handle timestamp queries
    elif 'START_TIMESTAMP' in sql.upper() or 'END_TIMESTAMP' in sql.upper():
        timestamps = [row[0] for row in results]  # Extract first element from each tuple
        explanation = "Timestamps:\\n" + "\\n".join(timestamps)

    # Handle generic queries
    else:
        if len(results[0]) == 1:  # Single column
            explanation = "Results:\\n" + "\\n".join([str(row[0]) for row in results])
        else:
            explanation = "Results:\\n" + "\\n".join([str(row) for row in results])

    return explanation

In [12]:
# Keep timestamps as strings throughout preprocessing
df['Start_Timestamp'] = df['Start_Timestamp'].astype(str)
df['End_Timestamp'] = df['End_Timestamp'].astype(str)

## SQL Accuracy function

In [13]:
from sqlalchemy import text

def check_sql_accuracy(temp_db, generated_sql, ground_truth_sql):
    """
    Compares the Gemini-generated SQL query with a manually written ground truth query.

    Parameters:
        temp_db (SQLAlchemy Engine): SQLite in-memory database.
        generated_sql (str): SQL query generated by Gemini.
        ground_truth_sql (str): Manually written correct SQL query.

    Returns:
        str: Accuracy message indicating if the query is correct or not.
    """
    try:
        with temp_db.connect() as conn:
            # Execute Gemini-generated SQL
            result_gemini = conn.execute(text(generated_sql)).fetchall()

            # Execute Ground Truth SQL
            result_ground_truth = conn.execute(text(ground_truth_sql)).fetchall()

            # Compare results
            if result_gemini == result_ground_truth:
                return "\n✅ The generated SQL is **accurate**!"
            else:
                return "\n❌ The generated SQL is **inaccurate**. Mismatch in results."

    except Exception as e:
        return f"\n❌ Error executing queries: {e}"

In [14]:
def convert_to_time(time_str):
    try:
        return pd.to_datetime(time_str, format='%H:%M:%S').time()
    except ValueError:
        return None

df['Start_Timestamp'] = df['Start_Timestamp'].apply(convert_to_time)
df['End_Timestamp'] = df['End_Timestamp'].apply(convert_to_time)

# Calculate duration, handling potential midnight crossings
def calculate_duration(start_time, end_time):
    if start_time is None or end_time is None:
        return None  # Handle missing values

    # Combine with a dummy date for comparison
    start_dt = pd.Timestamp.combine(pd.Timestamp.today(), start_time)
    end_dt = pd.Timestamp.combine(pd.Timestamp.today(), end_time)

    if end_dt < start_dt:
        end_dt += pd.Timedelta(days=1)

    return (end_dt - start_dt).total_seconds()

df['Duration'] = df.apply(lambda row: calculate_duration(row['Start_Timestamp'], row['End_Timestamp']), axis=1)

## Feature Engineering

In [15]:
df['Start_Timestamp'] = pd.to_datetime(df['Start_Timestamp'], format='%H:%M:%S')
df['End_Timestamp'] = pd.to_datetime(df['End_Timestamp'], format='%H:%M:%S')
df['Duration'] = (df['End_Timestamp'] - df['Start_Timestamp']).dt.total_seconds()

# Main Interaction Loop

In [ ]:
# Main interaction loop
while True:
    user_question = input("\nAsk a question (or 'exit'): ").strip()
    if user_question.lower() == 'exit':
        break

    # Generate SQL with validation
    try:
        sql = generate_sql(user_question)
        if not validate_sql(sql):
            raise ValueError("Generated SQL failed validation checks")

        print(f"\nGenerated SQL: {sql}")

        # Execute SQL
        results = execute_sql(sql)
        if results is None:  # No results found
            print("\n🔍 No matching records found in the dataset.")
        elif isinstance(results, str):  # Error occurred
            print(f"\n❌ {results}")
        else:
            # Explain results
            explanation = explain_results(results, user_question, sql)
            print(f"\n🔍 Analysis: {explanation}")

    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        print("Tip: Try rephrasing with facility names from: " + ", ".join(df['Facility'].unique()))


Ask a question (or 'exit'): Time duration of Article 1 at Vikaspuri

Generated SQL: SELECT End_Timestamp - Start_Timestamp AS Time_Duration
FROM articles
WHERE Article_ID = 1 AND Facility = 'Vikaspuri';

❌ Error: sequence item 0: expected str instance, int found
Tip: Try rephrasing with facility names from: Lajpat Nagar, Vikaspuri, Saket, Rohini, Janakpuri, Shadipur, Karol Bagh, Pitampura, Rajouri Garden, saket
